In [16]:
import pandas as pd
import numpy as np

data = pd.read_csv('diabetes.csv')
df = data.copy()
import numpy as np

In [17]:
df.head(10)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1
9,8,125,96,0,0,0.0,0.232,54,1


In [18]:
df.shape

(768, 9)

In [19]:
data.isnull().values.any()

False

In [20]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [30]:

from sklearn.model_selection import train_test_split

y = df.Outcome.values
X = data.drop(['Outcome'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=27)

In [22]:
# Checking to see if there are any missing values that were filled in as zero
def checkzero():
    columns = ['Glucose', 'BloodPressure', 'Insulin',
        'BMI']
    for col in columns:
        print('\n' + str(col) + ' Zero Values:')
        print(len(df.loc[df[str(col)] == 0]))
checkzero()


Glucose Zero Values:
5

BloodPressure Zero Values:
35

Insulin Zero Values:
374

BMI Zero Values:
11


In [31]:
# Now that I know there are still missing values, I need to imptue them. I will come back and hyperparameter this to got the best results.
from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=0, strategy="mean")
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

In [40]:
# building the model
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier(random_state=10)
random_forest_model.fit(X_train, y_train.ravel())

RandomForestClassifier(random_state=10)

In [41]:
predictions = random_forest_model.predict(X_test)

from sklearn import metrics

print("Accuracy = {0:.3f}".format(metrics.accuracy_score(y_test, predictions)))

Accuracy = 0.747


In [43]:
# Not bad accuracy, but I can do hyperparameters to make this even better.
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, 
              max_depth = max_depth,  
              min_samples_split = min_samples_split, 
              min_samples_leaf = min_samples_leaf)

rs = RandomizedSearchCV(random_forest_model, hyperF, cv=4, scoring='accuracy',verbose=10)
rs.fit(X_train, y_train) 

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8 
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8, score=0.792, total=   1.3s
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8 
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8, score=0.799, total=   1.3s
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8 
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.6s remaining:    0.0s
[CV]  n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8, score=0.752, total=   1.3s
[CV] n_estimators=800, min_samples_split=5, min_samples_leaf=5, max_depth=8 
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.0s remaining:    

RandomizedSearchCV(cv=4, estimator=RandomForestClassifier(random_state=10),
                   param_distributions={'max_depth': [5, 8, 15, 25, 30],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15,
                                                              100],
                                        'n_estimators': [100, 300, 500, 800,
                                                         1200]},
                   scoring='accuracy', verbose=10)

In [44]:
rs.best_score_

0.7833163568457686

In [49]:
rs.best_params_

{'n_estimators': 800,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_depth': 25}

In [50]:
# this will be the model that I will use.
rs.best_estimator_

RandomForestClassifier(max_depth=25, min_samples_leaf=2, n_estimators=800,
                       random_state=10)

In [61]:
# building the improved model
rfm = RandomForestClassifier(max_depth=25, min_samples_leaf=2, n_estimators=800,
                       random_state=10)
rfm.fit(X_train, y_train.ravel())

preds = rfm.predict(X_test)

from sklearn.model_selection import cross_val_score

score = cross_val_score(rfm, X_test, y_test.ravel(), cv = 10)
score.mean()

0.7770833333333333